# Sistemas Inteligentes 2021/2022

### Grupo 02: 
   - Maryny Zara Castada Collado
   - Seilys Iglesias Rojas


## Práctica 1. Parte IV. Muelle de carga 


#### a) Formalizar la representación de estados y operadores usando AIMA de la forma más general posible para el estado inicial y objetivo del ejemplo de la figura.

#### REPRESENTACIÓN DE ESTADOS:
La representación de estados consiste en una tupla de N elementos que representa la informacíon de cada contenedor (donde la posición 0 representa la pila 1, la siguiente representa la pila 2, y así sucesivamente) formada por un par de tuplas: 
- El primer elemento del par representa el número de la pila en la que se encuentra el contenedor.
- El segundo elemento representa la posición en la que se encuentra el contenedor en esa pila, siendo 0 la posición base y -1 la posición de la cima, Si en una pila solo hay un elemento entonces tendrá el valor -1.

A continuación se muestra el estado inicial y final con la representación que se ha explicado anteriormente para el ejemplo que se nos ha planteado:
- **Estado inicial**: ((1,0), (5,-1), (5,1), (4,0), (5,0), (6,-1), (1,-1), (4,-1), (2,0), (2,-1), (3,-1))
- **Estado final**: Contenedores 3,4 y 7 en el muelle objetivo (muelle 1) y que dichos contenedores no tengan encima un contenedor no objetivo.

#### VARIABLES AUXILIARES:
- Qué pilas corresponden a cada muelle, para el ejemplo planteado tendríamos lo siguiente:
     - **muelle(1,1,1,2,2,2)**
         - Es decir, las pilas 1,2 y 3 están en el muelle 1 (muelle objetivo), y las pilas 4,5 y 6 en el muelle 2.
- Altura máxima de cada muelle, supongamos una altura máxima para las pilas de ambos muelles de 3:
    - **altura_max(3,3)**

#### ACCIÓN:
- Solo tendríamos un única acción para este problema que sería mover un contenedor de una pila origen a una pila destino:
    - **mover(pila_origen, pila_destino)**
- Con esta representación y mediante las variables auxiliares que se han definido, se puede sacar en qué muelle se encuentra el contenedor, qué grúa mueve dicho contenedor y si se usa una cinta transportadora.

#### OPERADORES: 
El número de operadores (factor de ramificación) sería el número de pilas posibles a las que puede apilarse un contenedor.
Para el caso del ejemplo, tendríamos un factor de ramificación de 30 (6 contenedores x 5 pilas a las que puede apilarse el contenedor).

#### COSTE:
- Si un contenedor se mueve a una pila destino que está en el mismo muelle que la pila origen: **Coste 1**
- Si un contenedor se mueve a una pila destino que no está en el mismo muelle que la pila origen:
    - Grúa correspondiente coge contenedor de pila origen + 
    - Grúa correspondiente pone contenedor en la cinta transpotadora + 
    - Grúa correspondiente coge contenedor de la pila transportadora + 
    - Grúa correspondiente pone contenedor en pila destino = **Coste 4**

#### b) ¿Qué algoritmos de los vistos en clase utilizarías para resolver el problema? Razona la respuesta.

En este caso, al ser el factor de ramificación muy grande (30), no es adecuado utilizar el algoritmo de búsqueda primero en anchura.


(Yo creo que la mejor es búsqueda en profundidad con graph search, habría que mirarlo)


#### c) Resuelve el problema razonando convenientemente el comportamiendo de distintos algoritmos (a elegir por vosotros).

In [139]:
from search import Problem
import sys
sys.path.append('aima/')
import search

In [141]:
class ProblemaPuerto(Problem):
    
    
    def __init__(self, initial, goal=None):        
        Problem.__init__(self, initial, goal)
    
    
    # Función que define las pilas a las que puede apilarse un contenedor
    def actions(self, state):
        # Por ejemplo para el estado inicial: ((1,0), (5,-1), (5,1), (4,0), (5,0), (6,-1), (1,-1), (4,-1), (2,0), (2,-1), (3,-1))
        # Las acciones posibles son: 
        #        { 1: [2, 3, 4, 6], 2: [1, 2, 3, 4, 6], 3: [1, 2, 3, 4, 6], 4: [1, 2, 3, 6], 5: [1, 2, 3, 4, 6],
        #          6: [1, 2, 3, 4], 7: [2, 3, 4, 6], 8: [1, 2, 3, 6], 9: [1, 3, 4, 6], 10: [1, 3, 4, 6], 11: [1, 2, 4, 6] }
        
        # Donde la clave corresponde al número de contenedor y el valor es la lista de pilas donde puede apilarse 
        # dicho contenedor
        
        a = {}
        
        for contenedor in state:                                
            if(contenedor[1] == -1):
                lista_pilas = []
                for pila in range (1,7):
                    if(pila != contenedor[0] and self._is_valid(state, pila)):
                        lista_pilas.append(pila)
                i = state.index(contenedor) + 1
                a[i] = lista_pilas
            
        return a
    
    # Función que muestra el resultado de realizar una acción a un estado dado
    def result(self, state, action):        
        res = ()
        i = state.index((action[0], -1))
        
        for contenedor in state:
            if(state.index(contenedor) == i):
                res = res + ((action[1], -1),)
            else:
                res = res + (contenedor,)
        
        return res
        
    # Función que cuenta el número de contenedores que hay en una pila dada
    def contenedores_en_pila(self, state, pila):
        n = 0
        
        for p in state:
            if(p[0] == pila):
                n += 1
        
        return n
        
    # Función que determina si una pila de un muelle correspondiente no ha llegado a su altura máxima y por tanto se puede 
    # apilar otro contenedor    
    def _is_valid(self, state, pila):
            
        pilas_muelles = (1,1,1,2,2,2) # Variable auxiliar que representa en qué muelle se encuentra pila
        altura_max = (3,3) # Variable auxiliar que representa la altura máxima de cada muelle
        
        muelle = pilas_muelles[pila-1]
                
        if (self.contenedores_en_pila(state, pila) == altura_max[muelle-1]):
            return False            
        return True
    
    def coste_de_aplicar_accion(self, state, action):
        pilas_muelles = (1,1,1,2,2,2)
        
        muelle_pila_origen = pilas_muelles[action[0]-1]
        muelle_pila_destino = pilas_muelles[action[1]-1]
        
        if(muelle_pila_origen == muelle_pila_destino):
            return 1
        else:
            return 4
    
    
    # Función que determina si se ha llegado a un estado objetivo o no
    # def goal_test(self,state):
        
                        
      #  return True
            
            
    
    # Falta definir heuristica y probar algoritmos de búsqueda 
    
    

In [142]:
puerto = ProblemaPuerto(((1,0), (5,-1), (5,1), (4,0), (5,0), (6,-1), (1,-1), (4,-1), (2,0), (2,-1), (3,-1)))
puerto.initial

((1, 0),
 (5, -1),
 (5, 1),
 (4, 0),
 (5, 0),
 (6, -1),
 (1, -1),
 (4, -1),
 (2, 0),
 (2, -1),
 (3, -1))

In [144]:
puerto.coste_de_aplicar_accion(puerto.initial, (1,3))

1

In [145]:
puerto.coste_de_aplicar_accion(puerto.initial, (1,5))

4

In [146]:
puerto.actions(puerto.initial)

{2: [1, 2, 3, 4, 6],
 6: [1, 2, 3, 4],
 7: [2, 3, 4, 6],
 8: [1, 2, 3, 6],
 10: [1, 3, 4, 6],
 11: [1, 2, 4, 6]}

In [109]:
test = puerto.result(puerto.initial, (5, 6))
test

((1, 0),
 (6, -1),
 (5, 1),
 (4, 0),
 (5, 0),
 (6, -1),
 (1, -1),
 (4, -1),
 (2, 0),
 (2, -1),
 (3, -1))

#### d) (OPCIONAL) Resolver otras instancias de problema cambiando la situación inicial y final.

#### e) (OPCIONAL) Modificar alguno de los siguientes datos y comentar cómo afecta a la resolución del problema.
- Modificar la altura de las pilas de los muelles. Se puede especificar alturas distintas para cada muelle pero siempre la misma altura para todas las pilas de un muelle.
- Variar el número de pilas.
- Incluir nuevos contenedores y/o contenedores objetivo.
- Incluir un muelle adicional.